In [14]:
import numpy as np
import xarray as xr
from netCDF4 import Dataset
import pandas as pd
import matplotlib.pyplot as plt
import netCDF4 as nc
import os
import glob
import re

In [2]:
folder_paths = ['/cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250317_094824', '/cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250317_104942', '/cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250317_102335']

chlor_a_data_list = []

for folder_path in folder_paths:
    print("new folder")
    nc_files = [f for f in os.listdir(folder_path) if f.endswith('.nc')]
    for file_name in nc_files:
        file_path = os.path.join(folder_path, file_name)
        
        dataset = xr.open_dataset(file_path)

        chlor_a_data_list.append(dataset['chlor_a'])

        dataset.close()

print("went through all files")

new folder
new folder
new folder
went through all files


In [11]:


folder_paths = [
    #'/cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250317_094824', 
    #'/cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250317_104942', 
    '/cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250317_102335'
]

chlor_a_data_list = []

# Iterate over each folder
for folder_path in folder_paths:
    print(f"Processing folder: {folder_path}")
    
    # List all .nc files in the folder
    nc_files = [f for f in os.listdir(folder_path) if f.endswith('.nc')]
    
    # Iterate over each file
    for file_name in nc_files:
        file_path = os.path.join(folder_path, file_name)
        
        # Open the NetCDF file
        dataset = xr.open_dataset(file_path)

        # Append the chlorophyll data (chlor_a) to the list
        chlor_a_data_list.append(dataset['chlor_a'])

        # Close the dataset to free resources
        dataset.close()

print("Completed reading all files")




Processing folder: /cluster/home/maikents/satellite_data/MODISA_L3m_CHL_R2022.0-20250317_102335
Completed reading all files


In [ ]:
# Concatenate the chlorophyll data across the time dimension
# Assuming that the time dimension exists in all datasets and is consistent
concatenated_chlor_a = xr.concat(chlor_a_data_list, dim='time')



In [12]:
chunk_size = 20  # Adjust this depending on your memory capacity

# Initialize a variable to store the concatenated data
concatenated_chlor_a = None

# Process data in chunks
for i in range(0, len(chlor_a_data_list), chunk_size):
    # Get the current chunk of data
    chunk = chlor_a_data_list[i:i+chunk_size]
    
    # Concatenate the current chunk
    chunk_concat = xr.concat(chunk, dim='time')
    
    # If this is the first chunk, initialize the concatenated dataset
    if concatenated_chlor_a is None:
        concatenated_chlor_a = chunk_concat
    else:
        # Otherwise, concatenate the result with the previous concatenation
        concatenated_chlor_a = xr.concat([concatenated_chlor_a, chunk_concat], dim='time')

    print(f"Processed chunk {i // chunk_size + 1} of {len(chlor_a_data_list) // chunk_size + 1}")



Processed chunk 1 of 18
Processed chunk 2 of 18
Processed chunk 3 of 18
Processed chunk 4 of 18
Processed chunk 5 of 18
Processed chunk 6 of 18
Processed chunk 7 of 18
Processed chunk 8 of 18
Processed chunk 9 of 18
Processed chunk 10 of 18
Processed chunk 11 of 18
Processed chunk 12 of 18
Processed chunk 13 of 18
Processed chunk 14 of 18
Processed chunk 15 of 18
Processed chunk 16 of 18
Processed chunk 17 of 18
Processed chunk 18 of 18


In [13]:

output_file1 = '/cluster/home/maikents/satellite_data/chunks/chlor3.nc'
concatenated_chlor_a.to_netcdf(output_file1)

In [15]:
cl1 = xr.open_dataset('/cluster/home/maikents/satellite_data/chunks/chlor1.nc')
cl2 = xr.open_dataset('/cluster/home/maikents/satellite_data/chunks/chlor2.nc')
cl3 = xr.open_dataset('/cluster/home/maikents/satellite_data/chunks/chlor3.nc')

In [ ]:
concatenated_chlor_a = xr.concat([cl1, cl2, cl3], dim='time')

In [ ]:
output_file = '/cluster/home/maikents/satellite_data/chunks/concatenated_chlor_a.nc'
concatenated_chlor_a.to_netcdf(output_file)

In [ ]:
# Now you can calculate the mean, percentiles, etc.
# For example, calculate the mean across time
mean_chl = concatenated_chlor_a.mean(dim='time')

# Calculate the 10th percentile (min) across time
percentile_10 = concatenated_chlor_a.quantile(0.1, dim='time')

# Calculate the 90th percentile (max) across time
percentile_90 = concatenated_chlor_a.quantile(0.9, dim='time')

# Combine the results into a new xarray Dataset
features = xr.Dataset({
    'mean_chl': mean_chl,
    'percentile_10': percentile_10,
    'percentile_90': percentile_90
})

# Print the dataset structure to verify
print(features)

# Optionally, save the features to a new NetCDF file
features.to_netcdf('/path/to/output/chlorophyll_features.nc')

print("Features (mean, 10th percentile, 90th percentile) saved to 'chlorophyll_features.nc'")

In [4]:
chunk_size = 10
n_chunks = int(np.ceil(len(chlor_a_data_list) / chunk_size))

print(n_chunks)


109


In [5]:
concatenated_result1 = None
for i in range(0,10):
    chunk = chlor_a_data_list[i * chunk_size: (i + 1) * chunk_size]

    chunk_concat = xr.concat(chunk, dim='time')

    if concatenated_result1 is None:
        concatenated_result1 = chunk_concat
    else:
        concatenated_result1 = xr.concat([concatenated_result1, chunk_concat], dim='time')
    print(f"chunk {i} done")


chunk 0 done
chunk 1 done
chunk 2 done
chunk 3 done
chunk 4 done
chunk 5 done
chunk 6 done
chunk 7 done
chunk 8 done
chunk 9 done


In [8]:
concatenated_result12 = None
for i in range(11,22):
    chunk = chlor_a_data_list[i * chunk_size: (i + 1) * chunk_size]

    chunk_concat = xr.concat(chunk, dim='time')

    if concatenated_result12 is None:
        concatenated_result12 = chunk_concat
    else:
        concatenated_result12 = xr.concat([concatenated_result12, chunk_concat], dim='time')
    print(f"chunk {i} done")


chunk 11 done
chunk 12 done
chunk 13 done
chunk 14 done
chunk 15 done
chunk 16 done
chunk 17 done
chunk 18 done
chunk 19 done
chunk 20 done
chunk 21 done


In [9]:
concatenated_result13 = None
for i in range(23,36):
    chunk = chlor_a_data_list[i * chunk_size: (i + 1) * chunk_size]

    chunk_concat = xr.concat(chunk, dim='time')

    if concatenated_result13 is None:
        concatenated_result13 = chunk_concat
    else:
        concatenated_result13 = xr.concat([concatenated_result13, chunk_concat], dim='time')
    print(f"chunk {i} done")


chunk 23 done
chunk 24 done
chunk 25 done
chunk 26 done
chunk 27 done
chunk 28 done
chunk 29 done
chunk 30 done
chunk 31 done
chunk 32 done
chunk 33 done
chunk 34 done
chunk 35 done


In [10]:
concatenated_result1 = xr.concat([concatenated_result1, concatenated_result12, concatenated_result13], dim='time')

In [ ]:
output_file1 = '/cluster/home/maikents/satellite_data/chunks/chunk1.nc'
concatenated_result1.to_netcdf(output_file1)

In [5]:
concatenated_result2 = None
for i in range(37,45):
    chunk = chlor_a_data_list[i * chunk_size: (i + 1) * chunk_size]

    chunk_concat = xr.concat(chunk, dim='time')

    if concatenated_result2 is None:
        concatenated_result2 = chunk_concat
    else:
        concatenated_result2 = xr.concat([concatenated_result2, chunk_concat], dim='time')


In [6]:
concatenated_result22 = None
for i in range(46,59):
    chunk = chlor_a_data_list[i * chunk_size: (i + 1) * chunk_size]

    chunk_concat = xr.concat(chunk, dim='time')

    if concatenated_result22 is None:
        concatenated_result22 = chunk_concat
    else:
        concatenated_result22 = xr.concat([concatenated_result22, chunk_concat], dim='time')


In [7]:
concatenated_result23 = None
for i in range(60,72):
    chunk = chlor_a_data_list[i * chunk_size: (i + 1) * chunk_size]

    chunk_concat = xr.concat(chunk, dim='time')

    if concatenated_result23 is None:
        concatenated_result23 = chunk_concat
    else:
        concatenated_result23 = xr.concat([concatenated_result23, chunk_concat], dim='time')

In [ ]:
output_file2 = '/cluster/home/maikents/satellite_data/chunks/chunk2.nc'
concatenated_result2.to_netcdf(output_file2)

In [ ]:
concatenated_result3 = None
for i in range(73,109):
    chunk = chlor_a_data_list[i * chunk_size: (i + 1) * chunk_size]

    chunk_concat = xr.concat(chunk, dim='time')

    if concatenated_result3 is None:
        concatenated_result3 = chunk_concat
    else:
        concatenated_result3 = xr.concat([concatenated_result3, chunk_concat], dim='time')


In [ ]:
output_file3 = '/cluster/home/maikents/satellite_data/chunks/chunk3.nc'
concatenated_result3.to_netcdf(output_file3)

In [ ]:
concatenated_result = xr.concat([concatenated_result1, concatenated_result2, concatenated_result3], dim='time')

In [ ]:
chl_time_avg = concatenated_result.mean(dim='time')
print("averaged data")


In [ ]:
output_file = '/cluster/home/maikents/satellite_data/chlorophyll_time_avg.nc'
chl_time_avg.to_netcdf(output_file)


print(f"Time-averaged chlorophyll data saved to {output_file}")

"""
The output of the time-averaged chlorophyll data will look like this:
Dimensions: (lat, lon)
Data: Each point on the grid will have the average chlorophyll concentration over the period you’re averaging (in this case, across three years).
"""